In [164]:
import pulp
import pandas

# Cost function

numLFA : ???
numLF: ???
numLE: ???


In [165]:
### Cost Function
 
def costFunction_bus(numLFA, numLF, numLE):
    return (numLFA * busData['LFA']['cost']) + (numLF * busData['LF']['cost']) + (numLE * busData['LE']['cost'])

battSize = 400 # in kWh
    
def costFunction_batt(numMP,numHP):
    return numMP*battData['MP']['price']*battSize*(battData['HP']['cycleLife']/battData['MP']['cycleLife']) + numHP*battData['HP']['price']*battSize 

def costFunction_port(numMP,numHP,route,timeslot):
    duration = data['timeslot_1'][route]['duration']
    frequency = data['timeslot_1'][route]['bus_frequency']
    busOutside = duration/frequency
    totNum = numMP + numHP
    if numMP == 0:
        numPortC = totNum - busOutside
        return numPortC*chargeData['C']['price']
    else:
        numPortB = totNum - busOutside
        return numPortB*chargeData['B']['price']
    
def costFunction_wages():
    return 0

def costFunction_charging():
    return 0

def costFunction_maintenance():
    return 0

#combinationVariable['LFA']['A']['MP']['num']

def totalCostFunction(combinationVariable,route,timeslot):
    
    # 1. Getting number of buses by type (LFA,LF,LE)
    numDict= {}
    numDict['LFA']=0
    numDict['LF']=0
    numDict['LE']=0
    for busKey in combinationVariable:
        for chargerType in combinationVariable[busKey]:
            for batType in combinationVariable[busKey][chargerType]:
                numDict[busKey] += combinationVariable[busKey][chargerType][batType]['num']
    
    # 2. Getting number of batteries in all buses by type (MP,HP)
    battDict= {}
    battDict['MP']=0
    battDict['HP']=0
    for busKey in combinationVariable:
        for chargerType in combinationVariable[busKey]:
            for battType in combinationVariable[busKey][chargerType]:
                battDict[battType] += combinationVariable[busKey][chargerType][battType]['num']
    
    # 1. Cost of buying Buses
    costBus = costFunction_bus(numDict['LFA'],numDict['LF'],numDict['LE'])
    
    # 2. Cost of buying Batteries in Buses
    costBatt = costFunction_batt(battDict['MP'],battDict['HP'])
    
    # 3. Cost of charging Ports
    costPort = costFunction_port(battDict['MP'],battDict['HP'],route,timeslot)
    
    return costBus+costBatt+costPort

In [171]:
### Initializing Data (routes and batteries)

# Bus Data
busData = {
    'LFA': {
        'cost' : 570000,
        'capacity' : 140
    },
    'LF':{
        'cost' : 390000,
        'capacity' : 92
    },
    'LE': {
        'cost' : 350000,
        'capacity' : 60
    }
}

# Route Data
data = {
    'timeslot_1': {
        'time': 240,
        
        'route_1' : {
            'passengers' : 700,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 30
        },
        
        'route_2' : {
            'passengers' : 2000,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        },
        'route_3' : {
            'passengers' : 2000,
            'bus_frequency' : 10,
            'distance' : 15,
            'duration' : 55
        },
        'route_4' : {
            'passengers' : 300,
            'bus_frequency' : 20,
            'distance' : 7,
            'duration' : 20
        },
        'route_5' : {
            'passengers' : 2000,
            'bus_frequency' : 10,
            'distance' : 6,
            'duration' : 20
        },
        'route_6' : {
            'passengers' : 1500,
            'bus_frequency' : 1500,
            'distance' : 10,
            'duration' : 60
        },
        'route_7': {
            'passengers' : 3300,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        },
        'route_8' : {
            'passengers' : 1600,
            'bus_frequency' : 20,
            'distance' : 25,
            'duration' : 70
        },
        'route_9' : {
            'passengers' : 600,
            'bus_frequency' : 30,
            'distance' : 35,
            'duration' : 60
        },
        'route_10' : {
            'passengers' : 1000,
            'bus_frequency' : 30,
            'distance' : 30,
            'duration' : 60
        },
        'route_11' : {
            'passengers' : 400,
            'bus_frequency' : 30,
            'distance' : 40,
            'duration' : 70
        },
        'route_12' : {
            'passengers' : 1000,
            'bus_frequency' : 30,
            'distance' : 35,
            'duration' : 60
        }

    },
    'timeslot_2': {
        'time': 360,
        
        'route_1' : {
            'passengers' : 700,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 30
        },
        
        'route_2' : {
            'passengers' : 300,
            'bus_frequency' : 20,
            'distance' : 10,
            'duration' : 40
        },
        'route_3' : {
            'passengers' : 300,
            'bus_frequency' : 20,
            'distance' : 15,
            'duration' : 55
        },
        'route_4' : {
            'passengers' : 200,
            'bus_frequency' : 30,
            'distance' : 7,
            'duration' : 20
        },
        'route_5' : {
            'passengers' : 300,
            'bus_frequency' : 10,
            'distance' : 6,
            'duration' : 20
        },
        'route_6' : {
            'passengers' : 200,
            'bus_frequency' : 20,
            'distance' : 20,
            'duration' : 60
        },
        'route_7': {
            'passengers' : 1000,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        },
        'route_8' : {
            'passengers' : 800,
            'bus_frequency' : 20,
            'distance' : 25,
            'duration' : 70
        },
        'route_9' : {
            'passengers' : 200,
            'bus_frequency' : 30,
            'distance' : 35,
            'duration' : 60
        },
        'route_10' : {
            'passengers' : 600,
            'bus_frequency' : 30,
            'distance' : 30,
            'duration' : 60
        },
        'route_11' : {
            'passengers' : 100,
            'bus_frequency' : 60,
            'distance' : 40,
            'duration' : 70
        },
        'route_12' : {
            'passengers' : 200,
            'bus_frequency' : 60,
            'distance' : 35,
            'duration' : 60
        }
        
    },
    'timeslot_3': {
        'time': 240,
        
        'route_1' : {
            'passengers' : 700,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 30
        },
        
        'route_2' : {
            'passengers' : 2000,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        },
        'route_3' : {
            'passengers' : 2000,
            'bus_frequency' : 10,
            'distance' : 15,
            'duration' : 55
        },
        'route_4' : {
            'passengers' : 300,
            'bus_frequency' : 20,
            'distance' : 7,
            'duration' : 20
        },
        'route_5' : {
            'passengers' : 2000,
            'bus_frequency' : 10,
            'distance' : 6,
            'duration' : 20
        },
        'route_6' : {
            'passengers' : 1500,
            'bus_frequency' : 1500,
            'distance' : 10,
            'duration' : 60
        },
        'route_7': {
            'passengers' : 3300,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        },
        'route_8' : {
            'passengers' : 1600,
            'bus_frequency' : 20,
            'distance' : 25,
            'duration' : 70
        },
        'route_9' : {
            'passengers' : 600,
            'bus_frequency' : 30,
            'distance' : 35,
            'duration' : 60
        },
        'route_10' : {
            'passengers' : 1000,
            'bus_frequency' : 30,
            'distance' : 30,
            'duration' : 60
        },
        'route_11' : {
            'passengers' : 400,
            'bus_frequency' : 30,
            'distance' : 40,
            'duration' : 70
        },
        'route_12' : {
            'passengers' : 1000,
            'bus_frequency' : 60,
            'distance' : 35,
            'duration' : 60
        }

    },
    'timeslot_4': {
        'time': 240,
        
        'route_1' : {
            'passengers' : 700,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 30
        },
        
        'route_2' : {
            'passengers' : 300,
            'bus_frequency' : 20,
            'distance' : 10,
            'duration' : 40
        },
        'route_3' : {
            'passengers' : 300,
            'bus_frequency' : 20,
            'distance' : 15,
            'duration' : 55
        },
        'route_4' : {
            'passengers' : 200,
            'bus_frequency' : 30,
            'distance' : 7,
            'duration' : 20
        },
        'route_5' : {
            'passengers' : 300,
            'bus_frequency' : 10,
            'distance' : 6,
            'duration' : 20
        },
        'route_6' : {
            'passengers' : 200,
            'bus_frequency' : 20,
            'distance' : 20,
            'duration' : 60
        },
        'route_7': {
            'passengers' : 1000,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        },
        'route_8' : {
            'passengers' : 800,
            'bus_frequency' : 20,
            'distance' : 25,
            'duration' : 70
        },
        'route_9' : {
            'passengers' : 200,
            'bus_frequency' : 30,
            'distance' : 35,
            'duration' : 60
        },
        'route_10' : {
            'passengers' : 600,
            'bus_frequency' : 30,
            'distance' : 30,
            'duration' : 60
        },
        'route_11' : {
            'passengers' : 100,
            'bus_frequency' : 60,
            'distance' : 40,
            'duration' : 70
        },
        'route_12' : {
            'passengers' : 200,
            'bus_frequency' : 30,
            'distance' : 35,
            'duration' : 60
        }
        
    },
    'timeslot_3': {
        'time': 240,
        
        'route_1' : {
            'passengers' : 700,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 30
        },
        
        'route_2' : {
            'passengers' : 2000,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        },
        'route_3' : {
            'passengers' : 2000,
            'bus_frequency' : 10,
            'distance' : 15,
            'duration' : 55
        },
        'route_4' : {
            'passengers' : 300,
            'bus_frequency' : 20,
            'distance' : 7,
            'duration' : 20
        },
        'route_5' : {
            'passengers' : 2000,
            'bus_frequency' : 10,
            'distance' : 6,
            'duration' : 20
        },
        'route_6' : {
            'passengers' : 1500,
            'bus_frequency' : 1500,
            'distance' : 10,
            'duration' : 60
        },
        'route_7': {
            'passengers' : 3300,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        },
        'route_8' : {
            'passengers' : 1600,
            'bus_frequency' : 20,
            'distance' : 25,
            'duration' : 70
        },
        'route_9' : {
            'passengers' : 600,
            'bus_frequency' : 30,
            'distance' : 35,
            'duration' : 60
        },
        'route_10' : {
            'passengers' : 1000,
            'bus_frequency' : 30,
            'distance' : 30,
            'duration' : 60
        },
        'route_11' : {
            'passengers' : 400,
            'bus_frequency' : 30,
            'distance' : 40,
            'duration' : 70
        },
        'route_12' : {
            'passengers' : 1000,
            'bus_frequency' : 60,
            'distance' : 35,
            'duration' : 60
        }

    },
    'timeslot_4': {
        'time': 240,
        
        'route_1' : {
            'passengers' : 700,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 30
        },
        
        'route_2' : {
            'passengers' : 300,
            'bus_frequency' : 20,
            'distance' : 10,
            'duration' : 40
        },
        'route_3' : {
            'passengers' : 300,
            'bus_frequency' : 20,
            'distance' : 15,
            'duration' : 55
        },
        'route_4' : {
            'passengers' : 200,
            'bus_frequency' : 30,
            'distance' : 7,
            'duration' : 20
        },
        'route_5' : {
            'passengers' : 300,
            'bus_frequency' : 10,
            'distance' : 6,
            'duration' : 20
        },
        'route_6' : {
            'passengers' : 200,
            'bus_frequency' : 20,
            'distance' : 20,
            'duration' : 60
        },
        'route_7': {
            'passengers' : 1000,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        },
        'route_8' : {
            'passengers' : 800,
            'bus_frequency' : 20,
            'distance' : 25,
            'duration' : 70
        },
        'route_9' : {
            'passengers' : 200,
            'bus_frequency' : 30,
            'distance' : 35,
            'duration' : 60
        },
        
        'route_10' : {
            'passengers' : 600,
            'bus_frequency' : 30,
            'distance' : 30,
            'duration' : 60
        },
        'route_11' : {
            'passengers' : 100,
            'bus_frequency' : 60,
            'distance' : 40,
            'duration' : 70
        },
        'route_12' : {
            'passengers' : 200,
            'bus_frequency' : 30,
            'distance' : 35,
            'duration' : 60
        }
        
    }
}

# Battery Data

battData = {
    'HP': {
        'energyDensity': 60,
        'operatingV': 600,
        'maxChargeCur': 625,
        'maxDischargeCur': 500,
        'cycleLife': 6500,
        'price': 1150 
    },
    'MP':{
        'energyDensity': 7,
        'operatingV': 600,
        'maxChargeCur': 450,
        'maxDischargeCur': 450,
        'cycleLife': 3000,
        'price': 720
    }
}


chargeData = {
    'A': {
        'chargeV': 500,
        'chargeCur': 60,
        'price': 30000
    },
    'B': {
        'chargeV': 555,
        'chargeCur': 450,
        'price': 155000
    },
    'C': {
        'chargeV': 500,
        'chargeCur': 900,
        'price': 260000
    }
}

In [167]:
### Calculating max appearance from timeslot, route, battType, chargeType



def chargeDepletion(route,battType):
    dur = data['timeslot_1'][route]['duration']/60 # in hours
    
    #Assume maximum discharge current and max operating V
    dischargeRate = battData[battType]['maxDischargeCur']*battData[battType]['operatingV']
    return dur*dischargeRate
    
def rechargeDuration(chargeDepleted,battType,chargeType):
    rechargeRate1 = chargeData[chargeType]['chargeV']*chargeData[chargeType]['chargeCur'] # Charge rate pushed by charging port
    rechargeRate2 = chargeData[chargeType]['chargeV']*battData[battType]['maxChargeCur'] # Charge rate accepted by battery
    
    actualRechargeRate = max(rechargeRate1,rechargeRate2) 
    rechargeDur = chargeDepleted/actualRechargeRate
    return rechargeDur
                                            
def maxAppearanceLFA(route,timeslot,numLFA,battType,chargeType):
    timeslotDuration = data[timeslot]['time']
    roundtripDuration = data[timeslot][route]['duration']  
    
    # Calculating recharging time
    chargeDepleted = chargeDepletion(route,battType)
    rechargeDur = rechargeDuration(chargeDepleted,battType,chargeType)
    
    # Maximum number of appearance = number of bus * (rountrip duration + charging duration after roundtrip)
    maxAppearLFA = numLFA*timeslotDuration/(roundtripDuration+rechargeDur)
                                            
    return maxAppearLFA

def maxAppearanceLF(route,timeslot,numLF,battType,chargeType):
    timeslotDuration = data[timeslot]['time']
    roundtripDuration = data[timeslot][route]['duration']
    
    # Calculating recharging time
    chargeDepleted = chargeDepletion(route,battType)
    rechargeDur = rechargeDuration(chargeDepleted,battType,chargeType)
    
    maxAppearLF = numLF*timeslotDuration/(roundtripDuration+rechargeDur)
    return maxAppearLF

def maxAppearanceLE(route,timeslot,numLE,battType,chargeType):
    timeslotDuration = data[timeslot]['time']
    roundtripDuration = data[timeslot][route]['duration']
    
    # Calculating recharging time
    chargeDepleted = chargeDepletion(route,battType)
    rechargeDur = rechargeDuration(chargeDepleted,battType,chargeType)
    
    maxAppearLE = numLE*timeslotDuration/(roundtripDuration+rechargeDur)
    return maxAppearLE

# Main Algorithm

optimization of the number of buses (cost of the buses)

constraint1 : The cheapest bus should appear as much as possible
constraint2 : The buses should have enough capacity to carry the predicted number of people

In [168]:
def fill():
    chargingType = dict()
    for key in chargeData:
        if key not in chargingType:
            chargingType[key] = fillBat()
    return chargingType

counter = 0
def fillBat():
    batEntry = dict()
    
    global counter
    
    for key in battData:
        
        batEntry[key] = dict()
        batEntry[key]['num'] = pulp.LpVariable("num" + str(counter), lowBound=0,cat="Integer")
        batEntry[key]['app'] = pulp.LpVariable("app" + str(counter), lowBound=0,cat="Integer")
        counter += 1      
        
    return batEntry


# Data structure :

combinationVariable[busType][chargerType][batType]['num' OR 'app']

In [172]:
### Linear Programming Optimization

def solveLP(route,timeslot):

    # Instantiate Problem, Declaring holder variables
    model = pulp.LpProblem("Cost Optimisation", pulp.LpMinimize)
    totalNumApp = 0
    totalCapacity = 0
    
    #Creating JSON structure for all the buses
    combinationVariable = dict()
    
    
    
    
    for bus in busData:
        if bus not in combinationVariable:
            combinationVariable[bus] = fill()
    
    # Removing options with zero business sense      
    del combinationVariable['LFA']['B']['HP']
    del combinationVariable['LFA']['C']['MP']
    del combinationVariable['LF']['B']['HP']
    del combinationVariable['LF']['C']['MP']
    del combinationVariable['LE']['B']['HP']
    del combinationVariable['LE']['C']['MP']
    
    for key in combinationVariable:
        if key == 'LFA':
            fct_to_call = maxAppearanceLFA
        elif key == 'LF':
            fct_to_call = maxAppearanceLF
        else:
            fct_to_call = maxAppearanceLE
            
        for chargerType in combinationVariable[key]:
            for batType in combinationVariable[key][chargerType]:
                
                # Constraint 1 (Number of bus appearances)
                num = combinationVariable[key][chargerType][batType]['num']
                app = combinationVariable[key][chargerType][batType]['app']
                model += app <= fct_to_call(route, timeslot, num, batType, chargerType)
            
                #Constraint 2 (passenger)    
                totalNumApp = totalNumApp + app
                totalCapacity = totalCapacity + (app * busData[key]['capacity'])

    # Objective function
    model += totalCostFunction(combinationVariable,route,timeslot), "Cost"

    # Constraint 1 (Number of bus appearances)
    model += totalNumApp >= data[timeslot]['time']/data[timeslot][route]['bus_frequency']
    
    # Constraint 2 (Enough Passenger Capacity)
    model += totalCapacity >= data[timeslot][route]['passengers']
    
    # Solving
    
    #print(model)
    
    model.solve()
    
    print('For ' + timeslot + ", " + route + ":")
    
    print('Status: ' + str(pulp.LpStatus[model.status]))

    print("\n Parsing Content \n")

    for bus in combinationVariable:
        for charger in combinationVariable[bus]:
            for bat in combinationVariable[bus][charger]:
                num = combinationVariable[bus][charger][bat]["num"]
                app = combinationVariable[bus][charger][bat]["app"]
                
                print ("busType:{} chargerType:{} batteryType{} = Num:{}  Appearance:{}".format(bus, charger, bat, num.varValue, app.varValue))
    print('Cost: ' + str(pulp.value(model.objective)))
    print('\n')


# Initializing Problems

Initialize Across Different Routes and Timeslots

In [173]:
listRoutes = ['route_1','route_2','route_3','route_4','route_5','route_6','route_7','route_8','route_9','route_10','route_11',
             'route_12']
listTimeslots = ['timeslot_1','timeslot_2','timeslot_3','timeslot_4']

for routes in listRoutes:
    for times in listTimeslots:
        solveLP(routes,times)


For timeslot_1, route_1:
Status: Optimal

 Parsing Content 

busType:LFA chargerType:A batteryTypeHP = Num:0.0  Appearance:0.0
busType:LFA chargerType:A batteryTypeMP = Num:0.0  Appearance:0.0
busType:LFA chargerType:B batteryTypeMP = Num:0.0  Appearance:0.0
busType:LFA chargerType:C batteryTypeHP = Num:0.0  Appearance:0.0
busType:LF chargerType:A batteryTypeHP = Num:0.0  Appearance:0.0
busType:LF chargerType:A batteryTypeMP = Num:0.0  Appearance:0.0
busType:LF chargerType:B batteryTypeMP = Num:0.0  Appearance:0.0
busType:LF chargerType:C batteryTypeHP = Num:0.0  Appearance:0.0
busType:LE chargerType:A batteryTypeHP = Num:0.0  Appearance:0.0
busType:LE chargerType:A batteryTypeMP = Num:0.0  Appearance:0.0
busType:LE chargerType:B batteryTypeMP = Num:0.0  Appearance:0.0
busType:LE chargerType:C batteryTypeHP = Num:4.0  Appearance:24.0
Cost: 3500000.0


For timeslot_2, route_1:
Status: Optimal

 Parsing Content 

busType:LFA chargerType:A batteryTypeHP = Num:0.0  Appearance:0.0
busType:L

KeyboardInterrupt: 